In [ ]:
%pip install sagemaker

## Datasets located in s3 bucket

In [ ]:
import sagemaker
from sagemaker.inputs import TrainingInput
bucket_name = "Your-Bucket-Name-Here"
datasets_name = "dataset-folder-name-here"
train_input = TrainingInput(s3_data = "s3://" + bucket_name + '/' + datasets_name + '/' + 'train')
valid_input = TrainingInput(s3_data = "s3://" + bucket_name + '/' + datasets_name + '/' + 'valid')

## Setting hyperparameters

In [ ]:
hyperparameters = {"batch_size": 16,
                    "epochs": 1,
                    "learning_rate": 0.001,
                    "model_name": "DenseNet121",
                    "num_classes": 8,
                    }

In [ ]:
#Set up local mode for sagemaker
sagemaker_local_session = sagemaker.local.LocalSession()

In [ ]:
from sagemaker.pytorch.estimator import PyTorch
from sagemaker import get_execution_role

execution_role = get_execution_role()
#Create the estimator object for PyTorch
image_uri="Image-URI-of-your-custom-container"
estimator = PyTorch(
    source_dir = "src", #directory inside the container
    entry_point = "train_mlflow.py", # training script
    framework_version = "1.12", #PyTorch Framework version, keep it same as used in default example
    py_version = "py38", # Compatible Python version to use
    image_uri = image_uri,
    instance_count = 1, #number of EC2 instances needed for training
    instance_type = "ml.g5.xlarge", #Type of EC2 instance/s needed for training Use 'local" for local mode
    role = execution_role, #Execution role used by training job
    hyperparameters=hyperparameters,
    dependencies=['src/requirements.txt'], #Dependencies needed for training
)

inputs = {"train":train_input, "test": valid_input}
#Start the training
estimator.fit(inputs)